In [1]:
from common import *
from structures import *

DAY = 14

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 14: Restroom Redoubt ---</h2><p>One of The Historians needs to use the bathroom; fortunately, you know there's a bathroom near an unvisited location on their list, and so you're all quickly teleported directly to the lobby of Easter Bunny Headquarters.</p>
<p>Unfortunately, EBHQ seems to have "improved" bathroom security <em>again</em> after your last <a href="/2016/day/2">visit</a>. The area outside the bathroom is swarming with robots!</p>
<p>To get The Historian safely to the bathroom, you'll need a way to predict where the robots will be in the future. Fortunately, they all seem to be moving on the tile floor in predictable <em>straight lines</em>.</p>
<p>You make a list (your puzzle input) of all of the robots' current <em>positions</em> (<code>p</code>) and <em>velocities</em> (<code>v</code>), one robot per line. For example:</p>
<pre><code>p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
</code></pre>
<p>Each robot's position is given as <code>p=x,y</code> where <code>x</code> represents the number of tiles the robot is from the left wall and <code>y</code> represents the number of tiles from the top wall (when viewed from above). So, a position of <code>p=0,0</code> means the robot is all the way in the top-left corner.</p>
<p>Each robot's velocity is given as <code>v=x,y</code> where <code>x</code> and <code>y</code> are given in <em>tiles per second</em>. Positive <code>x</code> means the robot is moving to the <em>right</em>, and positive <code>y</code> means the robot is moving <em>down</em>. So, a velocity of <code>v=1,-2</code> means that each second, the robot moves <code>1</code> tile to the right and <code>2</code> tiles up.</p>
<p>The robots outside the actual bathroom are in a space which is <code>101</code> tiles wide and <code>103</code> tiles tall (when viewed from above). However, in this example, the robots are in a space which is only <code>11</code> tiles wide and <code>7</code> tiles tall.</p>
<p>The robots are good at navigating over/under each other (due to a combination of springs, extendable legs, and quadcopters), so they can share the same tile and don't interact with each other. Visually, the number of robots on each tile in this example looks like this:</p>
<pre><code>1.12.......
...........
...........
......11.11
1.1........
.........1.
.......1...
</code></pre>
<p>These robots have a unique feature for maximum bathroom security: they can <em>teleport</em>. When a robot would run into an edge of the space they're in, they instead <em>teleport to the other side</em>, effectively wrapping around the edges. Here is what robot <code>p=2,4 v=2,-3</code> does for the first few seconds:</p>
<pre><code>Initial state:
...........
...........
...........
...........
..1........
...........
...........

After 1 second:
...........
....1......
...........
...........
...........
...........
...........

After 2 seconds:
...........
...........
...........
...........
...........
......1....
...........

After 3 seconds:
...........
...........
........1..
...........
...........
...........
...........

After 4 seconds:
...........
...........
...........
...........
...........
...........
..........1

After 5 seconds:
...........
...........
...........
.1.........
...........
...........
...........
</code></pre>
<p>The Historian can't wait much longer, so you don't have to simulate the robots for very long. Where will the robots be after <code>100</code> seconds?</p>
<p>In the above example, the number of robots on each tile after 100 seconds has elapsed looks like this:</p>
<pre><code>......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
</code></pre>
<p>To determine the safest area, count the <em>number of robots in each quadrant</em> after 100 seconds. Robots that are exactly in the middle (horizontally or vertically) don't count as being in any quadrant, so the only relevant robots are:</p>
<pre><code>..... 2..1.
..... .....
1.... .....
           
..... .....
...12 .....
.1... 1....
</code></pre>
<p>In this example, the quadrants contain <code>1</code>, <code>3</code>, <code>4</code>, and <code>1</code> robot. Multiplying these together gives a total <em>safety factor</em> of <code><em>12</em></code>.</p>
<p>Predict the motion of the robots in your list within a space which is <code>101</code> tiles wide and <code>103</code> tiles tall. <em>What will the safety factor be after exactly 100 seconds have elapsed?</em></p>
</article>

In [7]:
lines = '''p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3'''.splitlines()

In [22]:
lines = get_input_lines(DAY)

In [8]:
lines

['p=0,4 v=3,-3',
 'p=6,3 v=-1,-3',
 'p=10,3 v=-1,2',
 'p=2,0 v=2,-1',
 'p=0,0 v=1,3',
 'p=3,0 v=-2,-2',
 'p=7,6 v=-1,-3',
 'p=3,0 v=-1,-2',
 'p=9,3 v=2,3',
 'p=7,3 v=-1,2',
 'p=2,4 v=2,-3',
 'p=9,5 v=-3,-3']

In [37]:
# width = 11
# height = 7
width = 101
height = 103
initial = []
for line in lines:
    part1, part2 = line.split(' ')
    p_nums = part1.split('=')[1].split(',')
    v_nums = part2.split('=')[1].split(',')
    p = int(p_nums[0]), int(p_nums[1])
    v = int(v_nums[0]), int(v_nums[1])
    initial.append((p, v))

In [24]:
from collections import defaultdict


steps = 100
counts = defaultdict(int)
wmid = width // 2
hmid = height // 2
for p, v in initial:
    px, py = p
    vx, vy = v
    x = (px + vx * steps) % width
    y = (py + vy * steps) % height
    # print(x, y)
    if x == wmid or y == hmid:
        continue
    counts[(x > wmid, y > hmid)] += 1
print(counts.values())
result = 1
for v in counts.values():
    result *= v
result

dict_values([132, 111, 131, 119])


228410028

In [25]:
send_result(DAY, 1, result)

"That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two] (solved in 18:45)"

In [26]:
show_task(DAY, 2)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>During the bathroom break, someone notices that these robots seem awfully similar to ones built and used at the North Pole. If they're the same type of robots, they should have a hard-coded <span title="This puzzle was originally going to be about the motion of space rocks in a fictitious arcade game called Meteoroids, but we just had an arcade puzzle.">Easter egg</span>: very rarely, most of the robots should arrange themselves into <em>a picture of a Christmas tree</em>.</p>
<p><em>What is the fewest number of seconds that must elapse for the robots to display the Easter egg?</em></p>
</article>

In [ ]:

for i in range(1, 301):
    m = Map(h=height, w=width)
    for p, v in initial:
        px, py = p
        vx, vy = v
        x = (px + vx * i) % width
        y = (py + vy * i) % height
        m[y][x] = '#'
    print()
    print(i)
    m.show()

In [45]:
for steps in range(1, 20000):
    m = Map(h=height, w=width)
    for p, v in initial:
        px, py = p
        vx, vy = v
        x = (px + vx * steps) % width
        y = (py + vy * steps) % height
        m[y][x] = '#'
    counter = 0
    for y in range(height//3, height//3*2):
        for x in range(width//3, width//3*2):
            if m[y][x] == '#':
                counter += 1
    # m.show()
    if counter >= 150:
        print(steps, counter)

8258 292
18661 292


In [50]:
steps = 8258
m = Map(h=height, w=width)
for p, v in initial:
    px, py = p
    vx, vy = v
    x = (px + vx * steps) % width
    y = (py + vy * steps) % height
    m[y][x] = '#'
for y in range(height//3, height//3*2):
    line = ''
    for x in range(width//3, width//3*2):
        line += '# ' if m[y][x] == '#' else '. '
    print(line)

. . . # . # . . . . . . . . . . . . . . # . . . . . . . . . . . . 
. . . . . # . . . . . . . . . . . . . # # # . . . . . . . . . . . 
. . . . . # . . . . . . . . . . . . # # # # # . . . . . . . . . . 
. . . . . # . . . . . . . . . . . # # # # # # # . . . . . . . . . 
. . . . . # . . . . . . . . . . # # # # # # # # # . . . . . . . . 
. . . . . # . . . . . . . . . . . . # # # # # . . . . . . . . . . 
. . . . . # . . . . . . . . . . . # # # # # # # . . . . . . . . . 
. . . . . # . . . . . . . . . . # # # # # # # # # . . . . . . . . 
. . . . . # . . . . . . . . . # # # # # # # # # # # . . . . . . . 
. . . . . # . . . . . . . . # # # # # # # # # # # # # . . . . . . 
. . . . . # . . . . . . . . . . # # # # # # # # # . . . . . . . . 
. # . . . # . . . . . . . . . # # # # # # # # # # # . . . . . . . 
. . . . . # . . . . . . . . # # # # # # # # # # # # # . . . . . . 
. . . . . # . . . . . . . # # # # # # # # # # # # # # # . . . . . 
. . . . . # . . . . . . # # # # # # # # # # # # # # # # # . . 

In [48]:
send_result(DAY, 2, 8258)

"That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 14! (solved in 25:28)"